In [9]:
# Import
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup as bs
from splinter import Browser
from IPython.display import clear_output

In [10]:
# Set path to chromedriver for web scraping
executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [54]:
# Variables
file_name = "FFS MLB Sheet.xlsx"
auto_fill_url = "off"

#### OPTIONAL FOR TEAM SPECIFIC REPORTS ####
#ffs_teams_list = ['CIN', 'BAL']
ffs_teams_list = ['ARI','ATL','BAL','BOS','CHC','CWS','CIN','CLE','COL','DET','HOU','KC','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SD','SF','SEA','STL','TB','TEX','TOR','WAS']


In [84]:
# Import 40_man_rosters_manual.csv
protected_players = pd.read_csv(f"rule_v/40_man_rosters_manual.csv")

positions = ['C,','C ',' C ','1B','2B','3B','SS','OF','SP','RP','LHP','RHP',' P ','P ']
numbers = ['1','2','3','4','5','6','7','8','9','0']
special_char = ['(', ')', '$', '.', '[', ']', '/', ',', '+', '*', '-']
other = ['FA', 'NCT', ' M', '  m']

all_protected = []

# Loop through and format playername
for playername in protected_players['playername']:
    for x in ffs_teams_list:
        playername = playername.replace(x, '') 
    for x in positions:
        playername = playername.replace(x, '')
    for x in numbers:
        playername = playername.replace(x, '')
    for x in special_char:
        playername = playername.replace(x, '')
    playername = playername.strip()
    for x in other:
        playername = playername.replace(x, '')
       
    all_protected.append(playername.strip())

protected_players['formatted_name'] = all_protected
all_protected
#protected_players

['Tyler Stephenson',
 'Curt Casali',
 'Dustin Garneau',
 'Dan Vogelbach',
 'Seth Beer',
 'Chris Taylor',
 'Luis Guillorme',
 'Richieartin',
 'MasoncCoy',
 'Edwin Rios',
 'Corey Dickerson',
 'Shogo Akiyama',
 'Austin Dean',
 'Jorge Bonifacio',
 'Lamonte Wade',
 "Ka'ai Tom",
 'Mark Payton',
 'Casey Golden',
 'Madison Bumgarner',
 'Danny Duffy',
 'Gio Gonzalez',
 'JA Happ',
 'Johan Oviedo',
 'Chris Stratton',
 'Corey Oswalt',
 'Jake Newberry',
 'James Hoyt',
 'Edubray Ramos',
 'Randy Rosario',
 'Brandon Waddell',
 'Silvino Bracho',
 'Jeremy Beasley',
 'Zack Brown',
 'Michael Rucker',
 'Christian Vazquez',
 'Brandon Belt',
 'Chris Davis',
 'Cavan Biggio',
 'Yolmer Sanchez',
 'Joe Panik',
 'Matt Duffy',
 'Dexter Fowler',
 'Ian Desmond',
 'Andrew Stevenson',
 'Josearmolejos',
 'Corey Kluber',
 'Chris Archer',
 'Merrill Kelly',
 'Brett Anderson',
 'Johneans',
 'Tejay Antone',
 'Jaime Barria',
 'Jose Cisnero',
 'John Curtiss',
 'Andre Scrubb',
 'Sean Poppen',
 'Codi Heuer',
 'Joey Bart',
 'Lui

In [5]:
# Import FFS MLB Sheet
# Read Player Names
name_cols = "G"
current_year = "2020"

player_list = []

# Loop through each tab to pull list of player names
for team in ffs_teams_list:
    # Variables
    sheet = pd.read_excel(file_name, sheet_name=team, usecols=name_cols, skiprows=2)
    names = sheet['Player']
    
    # Remove empty rows
    nan_value = float("NaN")
    names.replace("", nan_value, inplace=True)
    names.dropna(inplace=True)
    
    # Write progress to console
    print(f"Processing data for {team}")
    
    # loop through each name in the sheet
    for name in names:
        
        # Remove FFS lingo from name (formatting)
        name = name.replace('RV', '')
        name = name.replace('R1', '')
        name = name.replace(team, '')
        name = name.strip()
        
        # Player Variables
        final_player_name = name
        final_player_dob = ""
        final_player_position = ""
                
        urlname = name.replace(' ','%20')
        url = f"https://www.baseball-reference.com/search/search.fcgi?search={urlname}"
        browser.visit(url)
        html = browser.html
        soup = bs(html, 'html.parser')
        
        # Find the title of the page to determine if you are getting the Search page or a Player page
        title = soup.findAll('title')[0].text
        
        # Search page
        if name not in title:
            # Variables
            player_names = []
            player_links = []
            for div in soup.findAll('div', {'class': 'search-item'}):
                for a in div.findAll('a'):
                    last_year = ""
                    # Link
                    link = a['href']
                    # Split text on whitespace (10)
                    entry = a.text.split('          ')
                    # Player name
                    player_name = entry[0].strip()
                    # Last year played
                    # If multiple years listed, choose last year
                    if '-' in entry[1]:
                        last_year = (entry[1].strip()).replace('(','').replace(')','').split('-')[1].strip()
                        # If only 1 year listed, choose that year
                    else:
                        last_year = (entry[1].strip()).replace('(','').replace(')','').strip()
                    # If last_year is equal to current_year (if player was active this year)
                    if last_year == current_year or last_year == str(int(current_year) - 1):
                        player_names.append(player_name)
                        player_links.append(link)
            # If multiple current player entries found, manually select the correct entry
            if len(player_links) > 1:
                choice = 1
                # Option - Manual Url matching
                if auto_fill_url == "off":
                    count = 0
                    for p in player_links:
                        count += 1
                        print(f"({count}) https://www.baseball-reference.com{p}")    
                    choice = input(f"Multiple entries for {name} {team}. Select:")
                # Set player url from multiple (default is first entry)
                url = f"https://www.baseball-reference.com{player_links[int(choice) - 1]}"
                browser.visit(url)
                html = browser.html
                soup = bs(html, 'html.parser')
        
        # All Players
        # DOB
        try:
            final_player_dob = soup.findAll(itemprop="birthDate")[0]['data-birth']
        except:
            print(f"Error retrieving DOB for {name}")
        
        # Main Position from Player Page
        try:
            pos = soup.findAll(id="meta")[0].findAll('p')[0]
            # Split text on colon
            pos = pos.text.split(':')
            pos = pos[1].strip()
            final_player_position = pos
        except:
            pass

        # Format DOB and find age > 25
        dob_year = final_player_dob.split('-')[0].strip()
        if dob_year != '':
            year25 = int(current_year) - 25
            if int(dob_year) <= year25:
                # Create player object
                player_dict = {
                    "name": final_player_name,
                    "dob": final_player_dob,
                    "owner": team
                }
                player_list.append(player_dict)
        else:
            # Create player object
            player_dict = {
                "name": final_player_name,
                "dob": final_player_dob,
                "owner": team
            }
            player_list.append(player_dict)
        
        # Clear console output
        clear_output(wait=True)
    
# Export as csv
df = pd.DataFrame(player_list)
df.to_csv(f"rule_v/rule_v.csv")

IndentationError: expected an indented block (<ipython-input-5-c134513c806f>, line 29)